### Artificial Neural Networks

#### Introduction

ANNs aim to simulate the processing in human brain by stringing together nodes of computation in a network. In human brain, the neurons are connected to other neurons by axons in a continuous manner. In other words, there is no real distinction among the neurons that a given neuron is connected to. In contrast, the ANNs compartmentalize the neurons into layers for computation. A feed-forward network arranges the layers in a sequential layers in which a neuron in a given layer is connected to neurons in layers preceding as well as succeeding it.

Each neuron in a layer has a weight associated with it linking it back to a specific neuron in the preceding layer. This weight is used to compute the values of feature vectors in "forward propagation". The same weights are used to compute the gradient and update the weights during "back propagation".

#### Mechanics of ANN implementation

We will leverage a high-level module called `keras` for writing the ANNs and performing all the analysis. *Keras* abstracts the details of a NN and lets us focus on analyzing the data. It has the ability to employ multiple *packages* for actual NN processing such as feed-forward and back-propagation computations. We will be using `tensorflow` for actual neural network processing.  This module has the ability to parallelize the linear algebra computations across `gpu`s if available on a machine. Other packages that can work with Keras are *Microsoft CNTK* and *Theano*.

We use the **eye state Detection** data from the UCI repository as previously discussed. The data is loaded into a *numpy* array and split into training and testing sets.

#### First Pass

Running a basic feed-forward network with `Keras` gives us the following results.

In [ ]:
from timeit import default_timer as timer

import numpy as np
from keras.layers import Dense, Dropout
from keras.models import Sequential
from sklearn.cross_validation import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler, RobustScaler, MaxAbsScaler, MinMaxScaler
from sklearn.utils import class_weight

# Variables, global
iterations = 200
batch_size = 1000
# Good: RobustScaler()
scalarX = MinMaxScaler()
train_file = "data\eye_state.csv"
LABEL_POS = 14
# Original: 256,128,20
INPUT_NEURONS = 64
HIDDEN_NEURONS = 32
HIDDEN_LAYERS = 1

# Functions
def readNp(file, scalarX, label_pos):
    data = np.loadtxt(file, delimiter=',', skiprows=1)
    X = data[:, 0:label_pos]
    Y = data[:, label_pos]
    scalarX = scalarX.fit(X)
    X = scalarX.transform(X)
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=7)
    Y_train = Y_train.astype(int)
    Y_test = Y_test.astype(int)
    return X_train, X_test, Y_train, Y_test

def define_model(input_dim, in_neurons, out_neurons, hidden_dim, num_hidden_layer, is_dropout, output_act,
                 other_act='relu'):
    model = Sequential()
    model.add(Dense(in_neurons, input_dim=input_dim, kernel_initializer='normal', activation=other_act))
    if is_dropout:
        model.add(Dropout(0.2))
    for i in range(num_hidden_layer):
        model.add(Dense(hidden_dim, kernel_initializer='normal', activation=other_act))
        if is_dropout:
            model.add(Dropout(0.2))
    model.add(Dense(out_neurons, kernel_initializer='normal', activation=output_act))
    return model
# End functions

# Step 1: Read data
X_train, X_test, Y_train, Y_test = readNp(train_file, scalarX, LABEL_POS)
print("X rows:{} features:{} Y rows:{}".format(X_train.shape[0], X_train.shape[1], Y_train.shape[0]))

# Step 2: Create model
model = define_model(X_train.shape[1], INPUT_NEURONS, 1, HIDDEN_NEURONS, HIDDEN_LAYERS, True, 'sigmoid')
model.summary()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# class_weights = class_weight.compute_class_weight('balanced', np.unique(Y_train), Y_train)
history = model.fit(X_train, Y_train, batch_size=batch_size, epochs=iterations, verbose=0)

# Step 3: Evaluation
start = timer()
score = model.evaluate(X_test, Y_test, verbose=0)
print(
    "Model evaluation done:{} sec.\n Test-> loss:{} accuracy:{}".format(round(timer() - start, 2), score[0], score[1]))
Y_pred = model.predict(X_test).astype(int)
matrix = confusion_matrix(Y_test, Y_pred)
print(matrix)
